# Visual Data Processing
**Bardia Khosravi, MD MPH MHPE**

by: Bardia Khosravi, MD MPH MHPE <a href="https://twitter.com/Khosravi_Bardia"><img width="16" src="https://i.ibb.co/J7Tpg3Q/twitter-logo-2429.png"></a>

*Mayo Clinic AI Lab, Department of Radiology, Mayo Clinic, Rochester, MN, US*

## 0. Data preparation 

Run the next cell to prepare this session's files.

In [ ]:
!git clone https://github.com/BardiaKh/UoC_ARS_VisualDataProcessing.git
!mv UoC_ARS_VisualDataProcessing/* .

## 1. Introduction

In this workshop, we work with three image handling libraries, namely Pillow, OpenCV, and skimage. Each library has its own pros and cons, which we will cover at the end of this session. But first, lets dive into what images are made up of.

In [ ]:
# @title Run this cell to visualize an image with its pixel values. { vertical-output: true, display-mode: "form" }
# @markdown *You can take a sneak peak at the code*.

import pandas as pd
import matplotlib as mpl

img = pd.read_csv("image.csv")
cmap = mpl.colors.LinearSegmentedColormap.from_list("bw", ["black", "white"])
img.style.background_gradient(cmap=cmap)

In a colored image, you have three different dimensions:


1.   The first dimension is the image height (H)
2.   The second dimension is the image width (W)
3.   The third dimension is the image channels (C)

In short, you can say the array is `H:W:C`.

Most computer displays show images as a collection of pixels, and each pixel creates color by mixing specific amounts of red, green, and blue light. These specific amounts are stored in three different channels when we save images.

<br>
<img src="https://i.ibb.co/CbgLynF/Image-Channels.png" alt="Figure 5" style='margin:auto' border="0">

<u><b>Figure 1.</b> RGB Image Coordination System ([source](https://towardsdatascience.com/understanding-images-with-skimage-python-b94d210afd23))</u><br><br>

So imagine that our image is made up of three layers each with a height of 512px and a width of 512px. These layers are stacked together to create an RGB image.

Usually natural images are 8-bit, meaning that each pixel value can range from 0-255.

## 2. Libraries

Before starting, we have to install the three libraries. These libraries are already installed on Colab, but to install these libraries in your local computer, you can copy and paste the following code in a new cell and run the cell:

```python
!pip install scikit-image opencv-python pillow
```

*Disclaimer:* Most of the images used for this section are created by an AI model called [Stable Diffusion](https://stability.ai) ([source](https://simplified.com/blog/ai-text-to-image/stable-diffusion-images/)).

**2.1. Pillow (PIL)**

This is a high-level library for image IO and simple manipulation. It is based on C, so it is faster than pyhton-based packages. You can open a file with PIL like this:

In [ ]:
from PIL import Image

raw_image = Image.open("samples/img_1.png")
raw_image

As you see, notebook environment supports displaying PIL images out of the box, i.e. without using any library (such as `matplotlib`). As we said, we images are multidimentional arrays, let's convert this PIL image to an array and inspect its characteristics:

In [ ]:
import numpy as np

img = np.array(raw_image)

print("Image shape: ", img.shape)
print("Image data type: ", img.dtype)

So in other words, we have an image with height of 667px, width of 1000px and three channels (red, green, and blue).
Let's visualize each channel independently. As we said previously, we need matplotlib library to show numpy arrays:

In [ ]:
import matplotlib.pyplot as plt

# this is a magic command to display the image in the notebook
%matplotlib inline

fig, ax = plt.subplots(1, 3, figsize=(21, 7))
ax[0].imshow(img[:, :, 0], cmap="Reds")
ax[0].set_title("Red channel")
ax[1].imshow(img[:, :, 1], cmap="Greens")
ax[1].set_title("Green channel")
ax[2].imshow(img[:, :, 2], cmap="Blues")
ax[2].set_title("Blue channel")
_ = [ax_.set_axis_off() for ax_ in ax]
plt.show()

Grayscale images have only one channel (or even zero channels). This means that when being displayed, the value for the three RGB channels would be the same to create different shades of gray. Let's convert this image to grayscale and save it using PIL:

In [ ]:
grayscale_image = img.mean(axis=2)
grayscale_image = Image.fromarray(grayscale_image.astype(np.uint8))
grayscale_image.save("grayscale_image.png")

Pillow allows for simple operations like rotation and flipping:

In [ ]:
raw_image.transpose(Image.ROTATE_90)

In [ ]:
raw_image.transpose(Image.FLIP_LEFT_RIGHT)

There are some other capabilities, but other libraries that we discuss will do a better job in them so let's check them out.

**2.2. Skimage**

This is the most *pythonic* image processing library, it has a very familiar API and will take care of most pre- and post-processings that we usually need for deep learning tasks.

In [ ]:
from skimage import io

raw_image = io.imread("samples/img_2.png")
raw_image

As you see, skimage ones images as arrays, so there is no need to convert it to numpy array. We can use the `matplotlib` library as we did before:

In [ ]:
plt.imshow(raw_image);
print("Image shape: ", raw_image.shape)

What is the 4th channel?

In [ ]:
raw_image = raw_image[:, :, :3]

Let's resize the image to half its size:

In [ ]:
from skimage.transform import resize

new_image_size = (raw_image.shape[0]//2, raw_image.shape[1]//2) # height, width
resized_image = resize(raw_image, new_image_size)
plt.imshow(resized_image);

Let's inspect the values of the image:

In [ ]:
print("Raw Image Range: ", raw_image.min(), raw_image.max())
print("Resized Image Range: ", resized_image.min(), resized_image.max())

You see that the image data type is converted to float, ranging from 0-1. This is a common practice to rescale image intensities (will be discussed in future sessions), but we should be careful when we want to save the image, as we have to convert the range back to 0-255:

In [ ]:
resized_image = (resized_image * 255).astype(np.uint8)
io.imsave("resized_image.png", resized_image)

**Practical Challenge 1**: As the images are arrays, write a function that gets a 100*100px crop from the center of the image:

In [ ]:
def center_crop_image(image):
    size = 100
    original_image = image.copy()
    ## START CODE HERE ## (≈ 1 line of code)



    ## END CODE HERE ##

    return image

cropped_image = center_crop_image(raw_image)
print("Cropped Image Shape: ", cropped_image.shape)
plt.imshow(cropped_image)

Padding is very similar to cropping an image. Essentially, when you want to pad an image, you create a larger image (with zeros on all pixels, i.e., black) and then *"paste"* the original image onto the new one.

**Practical Challenge 2**: Based on the abpve description, write a function that center pads an image for 100px. The original image should be the surrounded by 100 pixels of zeros on each side:

In [ ]:
def center_pad_image(image):
    margin_size = 100
    original_image = image.copy()    
    ## START CODE HERE ## (≈ 2 lines of code)

    # initialize the large image with zeros (np.zeros)
    large_image = 

    

    ## END CODE HERE ##

    return large_image

padded_image = center_pad_image(raw_image)
print("Padded Image Shape: ", padded_image.shape)
plt.imshow(padded_image)

Doing image transformations with skimage is much more intuitive. You can import that specific transform (`from skimage.transforms import ...`) and then use it as a function. A full list of supported transforms can be found [here](https://scikit-image.org/docs/stable/api/skimage.transform.html).

In [ ]:
from skimage.transform import rotate

rotated_image = rotate(raw_image, angle=45)
plt.imshow(rotated_image);
print("Rotated Image Range: ", rotated_image.min(), rotated_image.max())

Skimage also has a lot more capabilities like contour detection, contrast enhancement, labeling operations, etc. that we will not discuss in this session.

**2.3. OpenCV (cv2)**

Open CV is a library of programming functions mainly aimed at real-time computer vision. It is used for image processing, video capture and analysis including features like face detection and object detection. Compared to other libraries, OpenCV is more efficient in processing (as it is based on C) and has a large number of functions.

Openning an image is easy with openCV:

In [ ]:
import cv2

raw_image = cv2.imread("samples/img_3.jpg")
raw_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)
plt.imshow(raw_image);
print("Image shape: ", raw_image.shape)
print("Type: ", type(raw_image))

One important difference, is that when an image is read by openCV, the order of channels are blue, green, and red (BGR). That's why we need to convert it using `cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)`.

Let's try some of the things we did before for resizing and rotating:

In [ ]:
resized_image = cv2.resize(raw_image, (400, 200)) # width, height
plt.imshow(resized_image);
print("Image shape: ", resized_image.shape)
print("Image Range: ", resized_image.min(), resized_image.max())

In [ ]:
rotated_image = cv2.rotate(raw_image, cv2.ROTATE_90_CLOCKWISE)
plt.imshow(rotated_image);

For rotating by arbitrary values, you have to create an affine matrix as a *"formula"* on how to change pixel values. Something like this:

In [ ]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  
  return result

rotated_image = rotate_image(raw_image, 45)
plt.imshow(rotated_image);

Sometimes, you might need to create a grayscale image, you can convert the color scheme to `GRAY`:

In [ ]:
grayscale_image = cv2.cvtColor(raw_image, cv2.COLOR_RGB2GRAY)
plt.imshow(grayscale_image, cmap="gray");
print("Image shape: ", grayscale_image.shape)

In some cases, you might want to create a binary mask from an image, meaning to convert all pixels to either zero and one:

In [ ]:
def binarize_image(image):
    threshold, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return binary_image

binary_image = binarize_image(grayscale_image)
plt.imshow(binary_image, cmap="gray");

Finally, we can save the image to disk using:

In [ ]:
cv2.imwrite("binary_image.png", binary_image)

Videos are created by sequentially showing images one after the other. So we can open a single frame, manipulate it, and then put the frames backtogether.

In [ ]:
capture = cv2.VideoCapture("video.mp4")
new_capture = cv2.VideoWriter("new_video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (100, 100))
while True:
    rect, frame = capture.read()

    if not rect:
        break

    new_frame = cv2.resize(frame, (100, 100), cv2.INTER_AREA)
    new_capture.write(new_frame)


capture.release()
new_capture.release()

There are other libraries for video opening, like `imageio` but all they do is just open the video, and frame-by-frame processing should be done by either of these three libraries.

## 3. Final Challenge

When doing deep learning projects, we need to make the shape of all images the same, so our models be trained on one image resolution. Here we have 14 images in the `./samples` folder:

In [ ]:
from glob import glob

image_paths = sorted(glob("samples/*.*"))
print(image_paths)

What we need is to have a function, that gets an image path (string), and image size(integer) and:
1. Reads the file
2. Converts all images to **three channels**
3. Resizes the image in a way that the **aspect ratio is not changes**.
4. Pads the extra space around the image, so that **the image would be centered**.
5. Saves the image to disk in a new folder called `./new_samples` as **.png** format.

You can use a mix of the three libraries that we mentioned today.

In [ ]:
def standardize_image(image_path, image_size):
    ## START CODE HERE ## (≈ 20 lines of code)
    # 1. Read the image
    image = 
    # 2. Ensure the image is RGB

    # 3. Resize the image

    # 4. Pad the image

    # 5. Save the image

    ## END CODE HERE ##

    return image

In [ ]:
#Sanity Check
from skimage import io

new_path = [path.replace("samples", "new_samples") for path in image_paths]
for path in image_paths:
    standardize_image(image_path=path, image_size=256)
    new_path = path.replace("samples", "new_samples")
    new_path = new_path.replace("jpg", "png")
    new_image = io.imread(new_path)
    if new_image.shape != (256, 256, 3):
        print("Image shape is not correct for image: ", path)
        break
    if new_image.min() < 0 or new_image.max() > 255:
        print("Image range is not correct for image: ", path)
        break
else:
    print("All images are correct!")

Thank you for your attention!